### Imports

In [18]:
import yaml
import requests
import pandas as pd
from langchain_openai import ChatOpenAI
from ikigai import Ikigai
import json

In [19]:
ENVIRONMENT = yaml.load(
    open("project.yaml", "r"), Loader=yaml.FullLoader
)
IKIGAI_API_KEY = ENVIRONMENT["ikigai_api_key"]
OPENAI_API_KEY = ENVIRONMENT["openai_api_key"]

### Get the data/flow information

In [20]:
# Setup ikigai python client
ikigai = Ikigai(user_email="eliza@ikigailabs.io", api_key=IKIGAI_API_KEY)
apps = ikigai.apps()
# Pull from the functional demo v2 app for preliminary implementation
demo_app = apps["Agentic Interface Test"]

In [21]:
# TODO: this is finicky– when we incorporate it, need to try catch

# Get datasets
datasets = demo_app.datasets()
# Put the dataset into a dictionary where the keys are the dataset names
# and the values dictionaries have random sample of 5 rows
sampled_dataset_dict = dict()
for dataset_name in datasets:
    df = datasets[dataset_name].df()
    sampled_dataset_dict[dataset_name] = df.sample(n=min(5,len(df.index)), random_state=42)

In [23]:
# Define url to interact with ikigai apis
BASE_URL = "https://api.ikigailabs.io/"
headers = {
            'User': "eliza@ikigailabs.io",
            'Api-key': IKIGAI_API_KEY,
            'Content-Type': 'application/json'
        }
# Go through each flow in the demo app
flows = demo_app.flows()
pipeline_definitions = dict()
for flow in flows:
    pipeline_id = flows[flow].flow_id
    url = f"{BASE_URL}/component/get-pipeline?project_id={demo_app.app_id}&pipeline_id={pipeline_id}"
    pipeline_response = requests.request("GET", url, headers=headers)
    if pipeline_response.status_code!=200:
        print("Error getting pipeline")
    else:
        pipeline_definition = pipeline_response.json()
        pipeline_definitions[flow] = pipeline_definition

#### Gather baseline knowledge about UIDs, facets, models

In [24]:
model_specifics = pd.read_csv("data/model_specifics.csv")

In [25]:
model_uids = list(model_specifics["UID"].unique())

In [26]:
mapping = pd.read_csv("data/uid_to_name.csv")
uid_to_name = dict()
for i in range(len(mapping.index)):
    row = mapping.iloc[i]
    uid_to_name[row["UID"]] = row["Facet Name"]

In [27]:
facet_to_description = dict()
for i in range(len(mapping.index)):
    row = mapping.iloc[i]
    facet_to_description[row["Facet Name"]] = row["Description"]

#### Parse pipeline information to pass to llm

In [28]:
def parse_pipeline_to_flat_json(pipeline_json, uid_to_name):
    nodes = []
    inputs_map = {}  # maps destination facet to list of source facets

    # Build a mapping of destination → [sources]
    for arrow in pipeline_json['pipeline']['definition']['arrows']:
        source = f"Facet {arrow['source']}"
        dest = f"Facet {arrow['destination']}"
        inputs_map.setdefault(dest, []).append(source)

    # Parse each facet into a node
    for facet in pipeline_json['pipeline']['definition']['facets']:
        facet_id = f"Facet {facet['facet_id']}"
        facet_type = uid_to_name.get(facet['facet_uid'], 'Unknown')
        dataset_name = facet['arguments'].get('dataset_name', facet['name'])
        
        node = {
            "id": facet_id,
            "type": facet_type
        }

        if facet_id in inputs_map:
            node["inputs"] = inputs_map[facet_id]

        if dataset_name:
            node["dataset_name"] = dataset_name

        nodes.append(node)

    return nodes

In [29]:
final_flow_structure = dict()
# Go through each pipeline definition and make a json of the name -> json
for name, pipeline_definition in pipeline_definitions.items():
    final_flow_structure[name] = parse_pipeline_to_flat_json(pipeline_definition, uid_to_name)

In [30]:
final_flow_structure

{'Forecast Data': [{'id': 'Facet 0',
   'type': 'IMPORTED',
   'dataset_name': 'Sales'},
  {'id': 'Facet 1', 'type': 'AI_CAST', 'inputs': ['Facet 0']},
  {'id': 'Facet 2',
   'type': 'EXPORTED',
   'inputs': ['Facet 1'],
   'dataset_name': 'forecasts'},
  {'id': 'Facet 3',
   'type': 'EXPORTED',
   'inputs': ['Facet 1'],
   'dataset_name': 'metrics'}],
 'Test2': [{'id': 'Facet 0',
   'type': 'IMPORTED',
   'dataset_name': 'All scenario summary'},
  {'id': 'Facet 1', 'type': 'PYTHON', 'inputs': ['Facet 0']},
  {'id': 'Facet 2',
   'type': 'EXPORTED',
   'inputs': ['Facet 1'],
   'dataset_name': 'output'}],
 'Combine Info': [{'id': 'Facet 0',
   'type': 'IMPORTED',
   'dataset_name': 'forecasts'},
  {'id': 'Facet 1', 'type': 'IMPORTED', 'dataset_name': 'Sales'},
  {'id': 'Facet 2',
   'type': 'PYTHON',
   'inputs': ['Facet 0', 'Facet 1', 'Facet 4']},
  {'id': 'Facet 3',
   'type': 'EXPORTED',
   'inputs': ['Facet 2'],
   'dataset_name': 'full forecasts'},
  {'id': 'Facet 4', 'type': 'IMP

In [31]:
for item in final_flow_structure.values():
    print(item)

[{'id': 'Facet 0', 'type': 'IMPORTED', 'dataset_name': 'Sales'}, {'id': 'Facet 1', 'type': 'AI_CAST', 'inputs': ['Facet 0']}, {'id': 'Facet 2', 'type': 'EXPORTED', 'inputs': ['Facet 1'], 'dataset_name': 'forecasts'}, {'id': 'Facet 3', 'type': 'EXPORTED', 'inputs': ['Facet 1'], 'dataset_name': 'metrics'}]
[{'id': 'Facet 0', 'type': 'IMPORTED', 'dataset_name': 'All scenario summary'}, {'id': 'Facet 1', 'type': 'PYTHON', 'inputs': ['Facet 0']}, {'id': 'Facet 2', 'type': 'EXPORTED', 'inputs': ['Facet 1'], 'dataset_name': 'output'}]
[{'id': 'Facet 0', 'type': 'IMPORTED', 'dataset_name': 'forecasts'}, {'id': 'Facet 1', 'type': 'IMPORTED', 'dataset_name': 'Sales'}, {'id': 'Facet 2', 'type': 'PYTHON', 'inputs': ['Facet 0', 'Facet 1', 'Facet 4']}, {'id': 'Facet 3', 'type': 'EXPORTED', 'inputs': ['Facet 2'], 'dataset_name': 'full forecasts'}, {'id': 'Facet 4', 'type': 'IMPORTED', 'dataset_name': 'forecasts with aux'}, {'id': 'Facet 5', 'type': 'EXPORTED', 'inputs': ['Facet 2'], 'dataset_name': '

#### Get the relevant operation info

In [32]:
# First get unique facets used
facets = set()
for item in final_flow_structure.values():
    for value in item:
        facets.add(value['type'])
# Then create mapping
relevant_facets = dict()
for value in facets:
    relevant_facets[value] = facet_to_description[value]

In [33]:
relevant_facets

{'AI_CAST': 'Forecasting for time series data',
 'RENAME_COLUMNS': 'Rename specified columns',
 'IMPORTED': 'Import from platform',
 'EXPORTED': 'Export data to a file on the ikigai platform',
 'PYTHON': 'Input is created by python code'}

#### Now take the final flow structure and feed it to the llm 

In [35]:
# User query here!! 
# user_query = "What are the metrics like for the aux forecasts?"
user_query = "How do metrics for aux data compare in q1 vs q2?"

In [43]:
# Structure the final flow structure in terms of prompting
messages = []
system_context_string = f"""
You will be given a list of datasets. 
For each dataset you will be provided with a random sample of 5 rows of the dataframe.

You will also be given a json that explains the way these datasets are all connected.
Each key is the name of the flow (or dataset connection) and each value is a list of 
operations that happen inside of the flow. Often, the first action imports a specific dataset
in some way. Then the final action exports a new dataset. Both of which exist in the aformentioned
list of datasets Each item in the list of operations has an 'id' which is a unique identifier for the
operation. It has a 'type' which explains which operation is being done. It has 'inputs' which is a list 
of the ids of the inputs to that operation. It has a 'dataset_name' which is the name of the input or 
output dataset that is directly associated with that operation. They exist mostly for import and export
data operations.

You will be given a list of descriptions of the operations that are listed under 'type' in 
the json. These will provide further context on exactly what is being done to the dataset.

Understand the relationshipss between the datasets. Given a user query, be prepared to answer which
dataset would contain the answer to this query.
"""

project_specific_string = f"""
List of datasets
{sampled_dataset_dict}

Json explaining dataset connections:
{final_flow_structure}

Descriptions of operations:
{relevant_facets}

Remember, respond with the dataset that would contain the answer the user query.
Consider carefully how the datasets are used in the context of the connections. If a dataset
is not connected to others, it is less likely to be used.
Choose the name of dataset from this list: {sampled_dataset_dict.keys()}
Return a json in the following structure. {{ 'dataset': 'name of dataset' }}
"""

messages.append(('system', system_context_string + project_specific_string))

messages.append(('human', user_query ))

In [44]:
messages

[('system',
  "\nYou will be given a list of datasets. \nFor each dataset you will be provided with a random sample of 5 rows of the dataframe.\n\nYou will also be given a json that explains the way these datasets are all connected.\nEach key is the name of the flow (or dataset connection) and each value is a list of \noperations that happen inside of the flow. Often, the first action imports a specific dataset\nin some way. Then the final action exports a new dataset. Both of which exist in the aformentioned\nlist of datasets Each item in the list of operations has an 'id' which is a unique identifier for the\noperation. It has a 'type' which explains which operation is being done. It has 'inputs' which is a list \nof the ids of the inputs to that operation. It has a 'dataset_name' which is the name of the input or \noutput dataset that is directly associated with that operation. They exist mostly for import and export\ndata operations.\n\nYou will be given a list of descriptions of t

In [45]:
##### USE LLM HERE to create metadata for explaining each of the parts of the dataset
# maybe add more information about the dataset
# add the datatypes of each column
# if they are categorical, then tell the llm that these are the ok columns to filter by 
# If categorical, provide the categories to filter by
# also maybe be able to filter by numbers? and time? but llm would be better at generating that
# Provide the unique values for the columns

In [46]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_kwargs={"response_format": {"type": "json_object"}})
answer = llm.invoke(messages).content

In [47]:
dataset_name = json.loads(answer)['dataset']

### Once have the dataset, see if code can be written

In [48]:
dataset_name

'metrics1'

In [466]:
user_query

'How do the forecasts in q1 compare to the forecasts in q2?'

In [467]:
full_target_df = datasets[dataset_name].df()

In [468]:
sampled_dataset_dict[dataset_name]

,Identifier,Time,prediction_type,Quantity,model,model_ranking,real/pred,identifier_ranking,Region,Product,SKU,TotalRevenue,TotalCost
40864,CMSBP Pink 25K mAh_South,2018-10-28 00:00:00,real,74.0,real,real,real,224.0,South,CMSBP,CMSBP Pink 25K mAh,3234.0,2496.34
92156,FTH Pink Wired Buds_South,2021-04-11 00:00:00,real,105.0,real,real,real,145.0,South,FTH,FTH Pink Wired Buds,3906.0,2560.60
147256,CMSBP Pink 10K mAh_West,2023-12-03 00:00:00,real,150.0,real,real,real,193.0,West,CMSBP,CMSBP Pink 10K mAh,2310.0,1649.90
32057,CMSBP Pink 15K mAh_West,2018-05-27 00:00:00,real,99.0,real,real,real,77.0,West,CMSBP,CMSBP Pink 15K mAh,2952.0,2174.64
79155,SSEHDD Black 1500 GB_Northeast,2020-08-23 00:00:00,real,215.0,real,real,real,6.0,Northeast,SSEHDD,SSEHDD Black 1500 GB,6210.0,4608.00


In [469]:
#### RANDOM STUFF I'VE WRITTEN DOWN
"""
So if you do not see
split_type == 'aux' in the input, do NOT filter for it.
"""

"\nSo if you do not see\nsplit_type == 'aux' in the input, do NOT filter for it.\n"

In [470]:
# Configure messages to write code properly
messages = []
system_context_string = f"""
You will be given a question and a sample of a pandas dataframe.
Write code in python that takes in a dataframe and outputs numeric values that
are helpful in answering the question. The output should also contain some
additional relevant details. Ensure that you are careful of datatypes when 
writing code. Do not filter the dataframe in your code for values you do not 
explicitly see given in the dataframe sample below. Thank you.
Here is the dataframe: {sampled_dataset_dict[dataset_name]}
Return a function with appropriate docstring in the following format:
    ```python
        {{function body}}
    ```
"""
messages.append(("system", system_context_string))
messages.append(("human", user_query))

In [471]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model="gpt-4o")
answer = llm.invoke(messages).content

In [472]:
generated_code = answer.strip().removeprefix("```python\n").removesuffix("\n```")

In [473]:
generated_code

'import pandas as pd\n\ndef compare_forecasts_q1_q2(df):\n    """\n    Compares the total revenue and total cost forecasts for Q1 and Q2.\n    \n    Parameters:\n    df (pd.DataFrame): DataFrame containing columns \'Time\', \'TotalRevenue\', and \'TotalCost\'.\n    \n    Returns:\n    dict: A dictionary with total revenue and total cost for Q1 and Q2.\n    """\n    # Convert \'Time\' column to datetime\n    df[\'Time\'] = pd.to_datetime(df[\'Time\'])\n    \n    # Extract month from \'Time\' column\n    df[\'Month\'] = df[\'Time\'].dt.month\n    \n    # Define Q1 and Q2 months\n    q1_months = [1, 2, 3]\n    q2_months = [4, 5, 6]\n    \n    # Calculate total revenue and total cost for Q1\n    q1_data = df[df[\'Month\'].isin(q1_months)]\n    q1_total_revenue = q1_data[\'TotalRevenue\'].sum()\n    q1_total_cost = q1_data[\'TotalCost\'].sum()\n    \n    # Calculate total revenue and total cost for Q2\n    q2_data = df[df[\'Month\'].isin(q2_months)]\n    q2_total_revenue = q2_data[\'TotalRe

In [474]:
print(generated_code)

import pandas as pd

def compare_forecasts_q1_q2(df):
    """
    Compares the total revenue and total cost forecasts for Q1 and Q2.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing columns 'Time', 'TotalRevenue', and 'TotalCost'.
    
    Returns:
    dict: A dictionary with total revenue and total cost for Q1 and Q2.
    """
    # Convert 'Time' column to datetime
    df['Time'] = pd.to_datetime(df['Time'])
    
    # Extract month from 'Time' column
    df['Month'] = df['Time'].dt.month
    
    # Define Q1 and Q2 months
    q1_months = [1, 2, 3]
    q2_months = [4, 5, 6]
    
    # Calculate total revenue and total cost for Q1
    q1_data = df[df['Month'].isin(q1_months)]
    q1_total_revenue = q1_data['TotalRevenue'].sum()
    q1_total_cost = q1_data['TotalCost'].sum()
    
    # Calculate total revenue and total cost for Q2
    q2_data = df[df['Month'].isin(q2_months)]
    q2_total_revenue = q2_data['TotalRevenue'].sum()
    q2_total_cost = q2_data['TotalCost'].su

In [475]:
### Add a step here where the logs and user output has an english explanation of the code

In [476]:
import ast

def extract_function_names(code_str):
    """
    Parses a Python code string and returns a list of function names defined in it.
    """
    tree = ast.parse(code_str)
    return [node.name for node in tree.body if isinstance(node, ast.FunctionDef)]

In [477]:
local_vars = {}
exec(generated_code, globals(), local_vars)
try:
    function_output = local_vars[extract_function_names(generated_code)[0]]
except:
    print("Function generation didn't work")

In [478]:
try:
    output = function_output(full_target_df)
except:
    print("Running code on df didn't work")

In [479]:
output

{'Q1': {'TotalRevenue': 1152568862.0, 'TotalCost': 835387537.43},
 'Q2': {'TotalRevenue': 835358316.0, 'TotalCost': 605114069.8199999}}

### Feed in code, a sample of df and then the outputs to llm

In [480]:
# Configure messages to write code properly
messages = []
system_context_string = f"""
You will be given a dataframe, code that runs on this dataframe,
and the output of that code. You will use this output and information
about what the code does to answer the user query.
Dataframe sample: {sampled_dataset_dict[dataset_name]}
Code: {generated_code}
Output of code: {output}
Answer the user query with an explanation of how the numbers were generated.
"""
messages.append(("system", system_context_string))
messages.append(("human", user_query))

In [481]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY)
answer = llm.invoke(messages).content

In [482]:
print(answer)

The forecasts in Q1 and Q2 are compared based on the total revenue and total cost for each quarter.

For Q1:
- Total Revenue: $1,152,568,862.0
- Total Cost: $835,387,537.43

For Q2:
- Total Revenue: $835,358,316.0
- Total Cost: $605,114,069.82

Comparing the two quarters:
- Q1 had a higher total revenue ($1,152,568,862.0) compared to Q2 ($835,358,316.0).
- Q1 also had a higher total cost ($835,387,537.43) compared to Q2 ($605,114,069.82).

In summary, both total revenue and total cost were higher in Q1 compared to Q2.
